In [1]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

import time

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import json

# = = = = = = = = = = = = = = =

import pandas as pd

In [2]:
with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

api

'LXnMWcYQhipLAS7rImEzmZ3CkrU033FMha9cwVSngG4vbufTsAOCQQ=='

In [3]:
count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        resp = requests.get('https://api.messefrankfurt.com/service/esb_api/exhibitor-service/api/2.1/public/exhibitor/search?language=zh-CN&q=&orderBy=name&pageNumber=1&pageSize=10000&orSearchFallback=false&showJumpLabels=true&findEventVariable=AUTOMECHANIKASHANGHAI',
                            headers=get_header(**{'Apikey': api}),
                            proxies=get_proxy(),
                            timeout=(100, 1000))

        if resp.status_code == 200 or resp.status_code == 401:
            dict_ = resp.json()

            if resp.status_code == 401 and dict_['status'] == 401 and dict_['message'].strip() == 'API GW: Unauthorized':
                print(f'fuck api {api}')
                time.sleep(3)
                with open('./api.txt', 'r') as api_txt:
                    api = api_txt.read()
            else:
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

resp

1
2
3


<Response [200]>

In [4]:
dict_['result']['metaData']

{'hitsTotal': 6454, 'hitsPerPage': 10000, 'currentPage': 1}

In [5]:
list_dict_factory =  [dict_factory['exhibitor'] for dict_factory in dict_['result']['hits']]

list_dict_factory[0]

{'id': 'mf_2_a3pk1a04hc0g~mf_2_10000100202401_A3PK1A04HC0G_386858',
 'synId': 'mf_2_a3pk1a04hc0g',
 'rewriteId': '31-incorporated',
 'name': '浙江埃克斯轮胎配件有限公司',
 'shortDescription': None,
 'exhibition': {'id': 'AUTOMECHANIKASHANGHAI',
  'name': 'Automechanika Shanghai 2024',
  'synonyme': None,
  'exhibitionHall': [{'id': '7.2',
    'name': '7.2',
    'sortKey': '0000000018',
    'nameLabel': {'localId': 'HALLNAME_7.2_AUTOMECHANIKASHANGHAI',
     'labels': {'zh-TW': {'localId': 'HALLNAME_7.2_AUTOMECHANIKASHANGHAI',
       'groupType': 'HALLDATA',
       'text': '7.2',
       'language': 'zh-TW'},
      'zh-CN': {'localId': 'HALLNAME_7.2_AUTOMECHANIKASHANGHAI',
       'groupType': 'HALLDATA',
       'text': '7.2',
       'language': 'zh-CN'},
      'en-GB': {'localId': 'HALLNAME_7.2_AUTOMECHANIKASHANGHAI',
       'groupType': 'HALLDATA',
       'text': '7.2',
       'language': 'en-GB'}}},
    'categoryLabel': {'localId': 'HALLCATEGORY_7.2_AUTOMECHANIKASHANGHAI',
     'labels': {'zh-TW': {

In [6]:
list_dict = []
for i, dict_factory in tqdm(enumerate(list_dict_factory), total=len(list_dict_factory), desc='Progress', ncols=77):
    dict_ = {'No': i + 1}
    for key, value in dict_factory.items():
        key = key.strip()
        key = key[0].upper() + key[1:]
        if not value or key in ['Id', 'SynId', 'RewriteId', 'Products', 'Href', 'OtherExhibitions', 'ProfileLabels', 'Description', 'SortKey', 'SearchHighlight', 'ExhibitorPrograms']:
            continue

        if type(value) == dict:
            for key_2, value_2 in value.items():
                if not value_2:
                    if key == 'Products':
                        dict_['Json_Product'] = json.dumps({})
                    continue

                key_2 = key_2.strip()
                key_2 = key_2[0].upper() + key_2[1:]
                if key == 'Address':
                    if key_2 == 'Country':
                        value_2 = value_2['label']
                    dict_[f'【{key}】{key_2}'] = value_2.strip()
                elif key == 'Addressrdm':
                    if key_2 == 'FormatedAddress':
                        value_2 = value_2.replace('<br />', '，')
                    dict_[f'【{key}】{key_2}'] = value_2.strip()
        elif type(value) == str:
            if key == 'Logo':
                if value:
                    dict_[key] = f'https://exhibitorsearch.hk.messefrankfurt.com/images/medium/{value.strip()}'
            elif key == 'LastApprovalDate':
                dict_['Last Approval Date'] = value.strip()
            else:
                dict_[key] = value.strip()
        elif type(value) == bool:
            dict_[key] = value
        elif type(value) == list:
            if key == 'KeyWords':
                text = ''
                for j, word in enumerate(value):
                    text += f'{j + 1}. {word.strip()}\n'
                dict_['Key Word'] = text.strip()
            elif key == 'Categories':
                text = ''
                for category in value:
                    for dict_subcategory in category['subCategories']:
                        text += f'''{dict_subcategory['name'].strip()}\n'''
                dict_['Category'] = text.strip()
            elif key == 'Social':
                dict_social = {}
                for j, social in enumerate(value):
                    dict_social[str(j)] = {}
                    dict_social[str(j)]['[Social] No'] = j + 1
                    dict_social[str(j)]['[Social] Name'] = social['network'].strip()
                    dict_social[str(j)]['[Social] File'] = f'''https://exhibitorsearch.hk.messefrankfurt.com/images/original/{social['imageFile'].strip()}''' if social['imageFile'] else ''
                    dict_social[str(j)]['[Social] Url'] = social['url'].strip()
                dict_['Json_Social'] = json.dumps(dict_social)
            elif key == 'PresentationLinks':
                dict_['Url'] = f'''https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/{value[0]['exhibitorUrlRewrite']}.html'''
                if len(value[0]['pstands']) > 1:
                    text = ''
                    for j, pstands in enumerate(value[0]['pstands']):
                        text += f'''{j + 1}: {pstands['hallAndLevel'].strip()} {pstands['firstBoothNumber'].strip()}\n'''
                    dict_['Presentation'] = text.strip()
                else:
                    dict_['Presentation'] = f'''{value[0]['pstands'][0]['hallAndLevel'].strip()} {value[0]['pstands'][0]['firstBoothNumber'].strip()}'''
    list_dict.append(dict_)

list_dict

Progress: 100%|███████████████████████| 6454/6454 [00:00<00:00, 11728.49it/s]


[{'No': 1,
  'Name': '浙江埃克斯轮胎配件有限公司',
  '【Address】Street': 'Hangzhou Gulf Chemical Zone Shanyu Wei Yi Rd',
  '【Address】City': 'Shaoxing',
  '【Address】Tel': '+86 575 8928 0232',
  '【Address】Country': '中国',
  '【Address】Email': 'svandyke@31inc.com',
  '【Addressrdm】CompanyName': '浙江埃克斯轮胎配件有限公司',
  '【Addressrdm】Country': 'CHN',
  '【Addressrdm】Locality': 'Shaoxing',
  '【Addressrdm】StreetName': 'Wei Yi Rd',
  '【Addressrdm】DependentLocality': 'Hangzhou Gulf Chemical Zone Shanyu',
  '【Addressrdm】Type': 'POSTAL',
  '【Addressrdm】WritingSystem': 'HANS',
  '【Addressrdm】FormatedAddress': '浙江埃克斯轮胎配件有限公司 Shaoxing Hangzhou Gulf Chemical Zone Shanyu，Wei Yi Rd，中国',
  'Category': '9.3 轮胎 / 车轮的维修及处理（橡胶硫化、平衡处理、磨损维修、弹力剂、维修材料、工具、填充剂、处理材料）',
  'Homepage': 'www.31inc.com',
  'Last Approval Date': '2024-09-12T04:28:41',
  'Url': 'https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/31-incorporated.html',
  'Presentation': '7.2 A41'},
 {'No': 2,
  'Name': '4A Truck Part

In [7]:
df = pd.DataFrame(list_dict).sort_values(by=['No'],
                                         ascending=[True],
                                         ignore_index=True)

list_column = df.columns.tolist()

list_column.remove('No')
list_column.remove('Name')
list_column.remove('Presentation')
list_column.remove('Category')
list_column.remove('Key Word')
list_column.remove('Url')
list_column.remove('Homepage')
list_column.remove('Json_Social')
list_column.remove('Last Approval Date')

df = df[['No', 'Name', 'Presentation', 'Category', 'Key Word', 'Url', 'Homepage', 'Json_Social', 'Last Approval Date'] + sorted(list_column)].to_excel('./file/1.manufacturer.xlsx', index=False)

df